In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def get_rankings(item, df):
    
    # Empty dict for results
    item_count_dict = {}
    
    # For every tag in df
    for i in range(df.shape[1]):
        
        # Calculate % of cases that tagged the item
        val_counts = df.iloc[:,i].value_counts(normalize = True)
        if item in val_counts.index:
            item_counts = val_counts[item]
        else:
            item_counts = 0
        
        # Add score to dict
        item_count_dict["tag_{}".format(i)] = item_counts
    
    return item_count_dict
def get_most_pop_position(item, df):
    
    # Empty dict for results
    item_count_dict = {}
    
    # For every tag in df
    for i in range(df.shape[1]):
        
        # Calculate % of cases that tagged the item
        val_counts = df.iloc[:,i].value_counts(normalize = True)
        if item in val_counts.index:
            item_counts = val_counts[item]
        else:
            item_counts = 0
        
        # Add score to dict
        item_count_dict["tag_{}".format(i)] = item_counts
    
    return item_count_dict

In [2]:
readers = pd.read_csv('Data/readers.csv')
readers_books = pd.read_csv('Data/readers_books.csv')
books = pd.read_csv('Data/Books/all_books.csv')

In [4]:
books_genres = books.iloc[:,[1,10]]
books_genres['genres'] = books_genres.genres.apply(eval)

C:\Users\LAURAW~1\AppData\Local\Temp/ipykernel_14256/1244133396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_genres['genres'] = books_genres.genres.apply(eval)


In [7]:
books_genres

,book_id,genres
0,45023611,"[Romance, Contemporary, Romance > Contemporary..."
1,1087204,"[Sequential Art > Manga, Romance, Sequential A..."
2,52397,"[Fiction, Science Fiction, Science Fiction > D..."
3,33413128,"[Contemporary, Audiobook, Sports > Sports, Adu..."
4,22318578,"[Nonfiction, Self Help, Audiobook, Self Help >..."
...,...,...
3805,170448,"[Classics, Fiction, Science Fiction > Dystopia..."
3806,18335634,"[Fantasy, Young Adult, Romance, Fantasy > Para..."
3807,51740,"[Young Adult, Fiction, Realistic Fiction, Spor..."
3808,41880609,"[Fiction, Poetry, LGBT, Contemporary, LGBT > Q..."


In [9]:
readers_books.merge(books_genres, on='book_id', how='inner')

,review_id,reader_rating,started_at,read_at,book_id,book_nr_reviews,book_nr_pages,book_year,book_avg_rating,book_nr_ratings,reader_id,genres
0,4388459254,2,Sat Dec 11 00:00:00 -0800 2021,Tue Dec 14 15:43:32 -0800 2021,32571395,30830,360.0,NaN,4.03,448958,4622890,"[Mystery, Young Adult, Contemporary, Thriller,..."
1,3967017587,4,Tue Apr 27 11:39:41 -0700 2021,Sat May 01 12:06:57 -0700 2021,32571395,30830,360.0,NaN,4.03,448958,23304719,"[Mystery, Young Adult, Contemporary, Thriller,..."
2,3086801418,3,NaN,NaN,32571395,30830,360.0,NaN,4.03,448958,2924544,"[Mystery, Young Adult, Contemporary, Thriller,..."
3,3072544609,3,Wed Dec 04 08:46:38 -0800 2019,Thu Dec 05 04:48:39 -0800 2019,32571395,30830,360.0,NaN,4.03,448958,4404986,"[Mystery, Young Adult, Contemporary, Thriller,..."
4,3100270162,0,NaN,NaN,32571395,30830,360.0,NaN,4.03,448977,15273212,"[Mystery, Young Adult, Contemporary, Thriller,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
175306,46817897,5,NaN,NaN,34898,1835,96.0,2005.0,4.08,37506,1571315,"[Fantasy, Childrens, Fiction, Classics, Childr..."
175307,912458131,1,NaN,NaN,34898,1835,96.0,2005.0,4.08,37506,30473596,"[Fantasy, Childrens, Fiction, Classics, Childr..."
175308,2995237671,3,Sun Sep 29 17:09:02 -0700 2019,Thu Oct 17 08:17:30 -0700 2019,34898,1835,96.0,2005.0,4.08,37506,5868885,"[Fantasy, Childrens, Fiction, Classics, Childr..."
175309,629723447,5,NaN,NaN,34898,1836,96.0,2005.0,4.08,37507,9840483,"[Fantasy, Childrens, Fiction, Classics, Childr..."


In [10]:
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [11]:
genre_counts = to_1D(books_genres.genres).value_counts()
top_genres = genre_counts[:60].index
#genres_expanded_v1 = books_genres["genres"].apply(pd.Series)
#sns.barplot(y=genre_counts, x=genre_counts.index)

In [97]:
for genre_col in all_genres:
    books_genres[genre_col] = books_genres['genres'].str.contains("'" + genre_col + "'").astype('int')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   book_id  981 non-null    int64 
 1   genres   981 non-null    object
dtypes: int64(1), object(1)
memory usage: 15.5+ KB


#### Quantify Bias

In [ ]:
score = ({'doesnt read':0, 'reads little rates low':1, 'reads little rates high':2,
          'reads a lot rates low':3, 'reads a lot rates high':4}

#### My convoluted approach

In [63]:
def get_all_genres(string):
    all_genres = set()
    genres =  re.findall(r'\'(.*?)\'', string)
    for genre in genres:
        all_genres.add(genre)
    return all_genres

In [61]:
for genre_col in all_genres:
    books_genres[genre_col] = books_genres['genres'].str.contains("'" + genre_col + "'").astype('int')

/var/folders/tv/8qzgqvq554vgrs4zb6bwbwlm0000gn/T/ipykernel_20550/556243298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_genres[genre_col] = books_genres['genres'].str.contains("'" + genre_col + "'").astype('int')
/var/folders/tv/8qzgqvq554vgrs4zb6bwbwlm0000gn/T/ipykernel_20550/556243298.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  books_genres[genre_col] = books_genres['genres'].str.contains("'" + genre_col + "'").astype('int')


In [64]:
books_genres.columns[2:]

Index(['Christian Fiction', 'Historical > Victorian', 'Fantasy > Supernatural',
       'Literature > Asian Literature', 'Fantasy > Paranormal',
       'Biography Memoir', 'Inspirational', 'Cultural > India', 'Travel',
       'Book Club',
       ...
       'Gender', 'Sequential Art > Graphic Novels', 'Folklore > Monsters',
       'Dark', 'Historical Fiction > Historical Fantasy', 'Nonfiction',
       'Historical', 'Romance > Erotic Romance', 'World War II > Holocaust',
       'Mystery > Cozy Mystery'],
      dtype='object', length=248)

In [62]:
genre
for column in books_genres.columns[2:]:
    

,book_id,genres,Christian Fiction,Historical > Victorian,Fantasy > Supernatural,Literature > Asian Literature,Fantasy > Paranormal,Biography Memoir,Inspirational,Cultural > India,...,Gender,Sequential Art > Graphic Novels,Folklore > Monsters,Dark,Historical Fiction > Historical Fantasy,Nonfiction,Historical,Romance > Erotic Romance,World War II > Holocaust,Mystery > Cozy Mystery
0,45023611,"['Romance', 'Contemporary', 'Romance > Contemp...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33413128,"['Contemporary', 'Audiobook', 'Sports > Sports...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22318578,"['Nonfiction', 'Self Help', 'Audiobook', 'Self...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,13623150,"['Young Adult', 'Science Fiction > Dystopia', ...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,35068705,"['Fantasy', 'Fiction', 'Historical > Historica...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,7126,"['Classics', 'Fiction', 'Historical > Historic...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
977,231804,"['Classics', 'Young Adult', 'Fiction', 'Academ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
978,170448,"['Classics', 'Fiction', 'Science Fiction > Dys...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
979,18335634,"['Fantasy', 'Young Adult', 'Romance', 'Fantasy...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [28]:
import re

In [57]:
type(books_genres.iloc[0,1])

str

In [53]:
type(books_genres.iloc[50,1])

str